In [3]:
!pip3 install --upgrade --quiet langchain langchain-community chromadb 
!pip3 install --upgrade --quiet pypdf pandas streamlit sentence-transformers ollama


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Import necessary modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import Ollama  # Replace OpenAI with Ollama

import os
import streamlit as st  
import pandas as pd

In [5]:
from langchain_community.llms import Ollama

# Initialize FREE LLM (Mistral 7B via Ollama)
llm = Ollama(model="mistral")  # No API key needed!

/var/folders/nm/wzd5h81n06q072t_27tj1jgw0000gn/T/ipykernel_20147/4110874448.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")  # No API key needed!


In [11]:
# Open terminal and run this in myenv, to run ollama server

# ollama serve
# After that, you pulled the mistral model tou use in your project with ollama pull mistral

In [10]:
!ollama --version


ollama version is 0.6.3


In [12]:
# Invoke the model to generate a response
response = llm.invoke("Tell me a joke about cats")

print(response)  # Output the joke

 Why don't cats play poker in the jungle?

Because there's too many cheetahs! (Cheetahs are often mispronounced as "cheaters" due to their resemblance and speed, making them untrustworthy card players.)


In [14]:
# Processing term sheet pdf

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF document
loader = PyPDFLoader("CIIE Term Sheet Template.pdf")  # Replace with your actual file
pages = loader.load()

# ✅ Step 2: Split into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   # Each chunk is ~1500 characters
    chunk_overlap=200,  # Overlapping context for better continuity
    length_function=len,
    separators=["\n\n", "\n", " "]  # Try to keep paragraphs intact
)

chunks = text_splitter.split_documents(pages)

# ✅ Step 3: Inspect the Split Chunks
print(f"Total chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks[:3]):  # Print first 3 chunks
    print(f"Chunk {i+1}:\n{chunk.page_content}\n")
    print("=" * 50)

Ignoring wrong pointing object 11 0 (offset 0)


Total chunks: 28

Chunk 1:
1

Chunk 2:
  Investment conditions:           This Term Sheet is neither an offer nor a binding commitment to purchase shares.  Such a binding commitment will only be made pursuant toet”) summarizes the terms and conditions of a proposed financing of Surya Power Magic Private Limited by Infuse Capital (a Scheme of i3E Trust and advised by CIIE Initiatives).  This Term Sheet is a preliminary summary of certain basic terms and conditions of a proposed financing and, except as specifically provided to the contrary herein, under no circumstances shall this Term Sheet create or be deemed to constitute a legally binding commitment on any party.  Company: XX Technologies Private Limited  Sponsor:                                          XX - , the "Sponsor"  Key Persons :                                           XX, the “Key Persons”  Existing Shareholders XX holding equity shares of the Company. Their equity shareholding is set out in the Table below.  Investor: 

In [24]:
# Loading the embedding

# # ✅ Step 1: Import the required modules
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.evaluation import load_evaluator

# # ✅ Step 2: Define the free embedding function
# def get_embedding_function():
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     return embeddings

# # ✅ Step 3: Get embeddings
# embedding_function = get_embedding_function()
# test_vector = embedding_function.embed_query("cat")

# # ✅ Step 4: Load the evaluator (measures how similar embeddings are)
# evaluator = load_evaluator(
#     evaluator="embedding_distance", 
#     embeddings=embedding_function
# )

# # ✅ Step 5: Evaluate similarity between prediction & reference
# print(evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop"))
# print(evaluator.evaluate_strings(prediction="Paris", reference="coffeeshop"))

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ✅ Load PDF document
loader = PyPDFLoader("CIIE Term Sheet Template.pdf")  # Replace with your actual file
pages = loader.load()

# ✅ Split into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   
    chunk_overlap=200,  
    length_function=len,
    separators=["\n\n", "\n", " "]  
)

chunks = text_splitter.split_documents(pages)

# ✅ Check chunk count
print(f"Total chunks: {len(chunks)}")


Ignoring wrong pointing object 11 0 (offset 0)


Total chunks: 28


In [26]:
# Vector database
# import uuid
# from langchain.vectorstores import Chroma

# # ✅ Step 2: Create and Store Vector Embeddings
# def create_vectorstore(chunks, embedding_function, vectorstore_path):
#     # Create unique IDs for each chunk
#     unique_ids = set()
#     unique_chunks = []
    
#     for chunk in chunks:
#         chunk_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, chunk.page_content))
#         if chunk_id not in unique_ids:
#             unique_ids.add(chunk_id)
#             unique_chunks.append(chunk)

#     # ✅ Step 3: Store Chunks in ChromaDB
#     vectorstore = Chroma.from_documents(
#         documents=unique_chunks, 
#         embedding=embedding_function,  # Free embeddings
#         persist_directory=vectorstore_path
#     )

#     vectorstore.persist()  # Save data to disk
#     return vectorstore

# # ✅ Step 4: Create the Vectorstore (Saves Chunks for Retrieval)
# vectorstore = create_vectorstore(
#     chunks=chunks, 
#     embedding_function=embedding_function, 
#     vectorstore_path="vectorstore_test"
# )

# print("✅ Vectorstore successfully created & persisted!")

from langchain.vectorstores import Chroma
from langchain.schema import Document

# ✅ Initialize Vector Store
vectorstore = Chroma(
    persist_directory="vectorstore_chroma", 
    embedding_function=embedding_function  # Use the HuggingFace embeddings
)

# ✅ Convert chunks to Chroma format
documents = [Document(page_content=chunk.page_content) for chunk in chunks]

# ✅ Add Documents to Chroma DB
vectorstore.add_documents(documents)
vectorstore.persist()
print(f"Number of documents in vectorstore: {vectorstore._collection.count()}")



Number of documents in vectorstore: 56


In [18]:
!pip install -U langchain-chroma

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [27]:
# Query for relevant data

# ✅ Create Retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",  # ✅ Maximal Marginal Relevance
    search_kwargs={"k": 5}  # ✅ Retrieve top 5 most relevant chunks
)

# ✅ Query for Key Financing Terms
query = "What are the key financing terms in the term sheet?"
relevant_chunks = retriever.invoke(query)

# ✅ Print Retrieved Chunks
for i, chunk in enumerate(relevant_chunks):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n{'='*50}")


Chunk 1:
Legal Counsel and Fees: The Company will pay its own legal fees and will also pay the fees of legal counsel and accounting due diligence to the Investor. The expenses towards legal and accounting due diligence by the Investor shall initially be born by the Investor. The Company shall reimburse these expenses to the Investor only after successful completion of transaction and remittance of the full amount of financing as agreed herein. Such reimbursement is subject to the upper limit of Rs.400,000/-.   Expiration; No-Shop: This Term Sheet will expire at 5:00 p.m. IST, XX unless signed by the Company and the Investor by such date.  By signing this Term Sheet, and in consideration of the efforts of the Investor in evaluating the Company, the Company agrees not to, for the term of this Term Sheet, directly or indirectly initiate, solicit, encourage, discuss, negotiate or accept any offers or proposals regarding the sale of equity securities of the Company or a substantial portion 

In [28]:
# Prompt for the term sheet

PROMPT_TEMPLATE = """
You are a legal and financial document analysis assistant.  
Your task is to extract **clear, factual, and concise answers** from the provided term sheet.  

Use the following retrieved sections of the document to answer the question.  
If the information is not present, **explicitly state that the term sheet does not provide an answer**.  
**Do not assume, infer, or fabricate any details.**  

{context}  

---

**Question:** {question}  
**Answer based on the term sheet:**  
"""


In [29]:
# Generating responses

from langchain.prompts import ChatPromptTemplate

# ✅ Concatenate context text with clear separation
context_text = "\n\n---\n\n".join([doc.page_content.strip() for doc in relevant_chunks])

# ✅ Define a specialized prompt for term sheet extraction
PROMPT_TEMPLATE = """
You are a legal and financial document analysis assistant.  
Your task is to extract **clear, factual, and concise answers** from the provided term sheet.  

Use the following retrieved sections of the document to answer the question.  
If the information is not present, **explicitly state that the term sheet does not provide an answer**.  
**Do not assume, infer, or fabricate any details.**  

{context}  

---

**Question:** {question}  
**Answer based on the term sheet:**  
"""

# ✅ Create and format the prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What are the key financing terms in the term sheet?")

print(prompt)


Human: 
You are a legal and financial document analysis assistant.  
Your task is to extract **clear, factual, and concise answers** from the provided term sheet.  

Use the following retrieved sections of the document to answer the question.  
If the information is not present, **explicitly state that the term sheet does not provide an answer**.  
**Do not assume, infer, or fabricate any details.**  

Legal Counsel and Fees: The Company will pay its own legal fees and will also pay the fees of legal counsel and accounting due diligence to the Investor. The expenses towards legal and accounting due diligence by the Investor shall initially be born by the Investor. The Company shall reimburse these expenses to the Investor only after successful completion of transaction and remittance of the full amount of financing as agreed herein. Such reimbursement is subject to the upper limit of Rs.400,000/-.   Expiration; No-Shop: This Term Sheet will expire at 5:00 p.m. IST, XX unless signed by 

In [30]:
# Response Generation

import ollama

# ✅ Use Ollama to generate a response
response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])

# ✅ Print response
print(response["message"]["content"])  # Extracts text response

1. Type of Security: Convertible Preference Shares (the "Preference Shares"), initially convertible into an equal number of the Company's equity shares (the "Investor Shares").
2. Pre-money Valuation: [Redacted] ("Pre-money Valuation").
3. Amount of Financing: [Redacted], to be invested by Investor in 1 tranche.
4. The investment will be completed within 30 days after execution of this Term Sheet, provided the Company on or prior to [redacted] executes this Term Sheet. If not executed by such date, or any other extended date as mutually agreed upon, this investment proposal will be deemed withdrawn.
5. The Investor has completed, to its satisfaction the legal and accounting review, social and environmental review (if applicable), and approved the investment in accordance with its governance rules.
6. The Company has received all necessary approvals (corporate or otherwise) necessary to complete the contemplated transaction; and any other condition as may be stipulated in the Share Subs

In [32]:
# ✅ Define RAG Chain Using LangChain Expression Language (LCEL)

from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# ✅ Function to Format Retrieved Chunks
def format_docs(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

# ✅ Define Prompt Template
PROMPT_TEMPLATE = """
You are a legal and financial document analysis assistant.
Use the following retrieved context to answer the question.
If the information is not in the document, say you don't know.

{context}

---

**Question:** {question}
**Answer:**
"""
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

rag_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),  # Use `RunnableLambda`
        "question": RunnablePassthrough(),
    }
    | prompt_template
    | llm
)

# ✅ Invoke RAG Chain with Ollama
response = rag_chain.invoke("What are the key financing terms in the term sheet?")

# ✅ Print the Response
print(response)

1. Type of Security: The Investor will invest in Convertible Preference Shares, initially convertible into an equal number of equity shares of the Company.
2. Amount of Financing: The Investor will invest a total of Rs. XX (not specified in the provided context).
3. Pre-money Valuation: The pre-money valuation of the company is Rs. XX.
4. Capital Structure: The capital structure will be as set forth in the provided table.
5. Closing Date: The closing date for the transaction, if the Term Sheet is executed, is on or before a specified date, with an option for extension by mutual agreement.
6. Investment conditions: A binding commitment to purchase shares will only be made upon satisfaction of certain conditions, as listed in the provided context. These include allocation of shares for ESOP pool, full-time employment agreements with the Sponsor and Key Persons, completion of a social and environmental review by the Investor, among others.
7. Insurance Requirements: The Company is require

In [37]:
# Generate Structured Responses

import ollama
import json
import re
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama

# ✅ Load Ollama Model
llm = Ollama(model="mistral")  # Ensure you're using the correct model

# ✅ Define Dummy Retriever (Replace with your actual retriever)
def dummy_retriever(_):
    return ["Dummy term sheet text for testing."]

retriever = RunnableLambda(dummy_retriever)

# ✅ Function to Format Retrieved Chunks
def format_docs(docs):
    return "\n\n---\n\n".join(docs)

# ✅ Define Prompt Template for Term Sheet Extraction
PROMPT_TEMPLATE = """
You are an expert in analyzing financial term sheets. Extract the key investment details in a structured JSON format.

{context}

---

Extract the following details:
1. **Company Name**
2. **Investor Name**
3. **Type of Security** (e.g., Convertible Preference Shares, Equity)
4. **Pre-money Valuation**
5. **Financing Amount**
6. **Investment Tranches** (Number of stages in which funds are disbursed)
7. **Key Investment Conditions**
8. **Equity Shareholding Information**
9. **Voting Rights & Governance Terms**
10. **Liquidation Preferences**

Provide ONLY the JSON output, without any explanations:

```json
{{
    "company_name": "Extracted Company Name",
    "investor_name": "Extracted Investor Name",
    "security_type": "Extracted Security Type",
    "pre_money_valuation": "Extracted Pre-money Valuation",
    "financing_amount": "Extracted Financing Amount",
    "investment_tranches": "Extracted Investment Tranches",
    "key_conditions": "Extracted Key Investment Conditions",
    "equity_shareholding": "Extracted Equity Shareholding Information",
    "governance_terms": "Extracted Governance Terms",
    "liquidation_preferences": "Extracted Liquidation Preferences"
}}
```
"""
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

# ✅ Define RAG Chain
rag_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),
    }
    | prompt_template
    | llm  # ❌ Removed `.with_structured_output()` since it's not supported
)

# ✅ Invoke RAG Chain with Ollama
response = rag_chain.invoke("Extract key financial details from the term sheet.")

# ✅ Extract JSON from Response
json_match = re.search(r"```json\n(.*?)\n```", response, re.DOTALL)

if json_match:
    json_text = json_match.group(1)  # Extract JSON content
    try:
        extracted_data = json.loads(json_text)  # Parse JSON
        print(json.dumps(extracted_data, indent=4))
    except json.JSONDecodeError:
        print("Error: The extracted JSON is not valid.")
        print(json_text)
else:
    print("Error: No valid JSON found in the response.")
    print(response)



{
    "company_name": "Dummy Company Name",
    "investor_name": "Dummy Investor Name",
    "security_type": "Dummy Security Type",
    "pre_money_valuation": "Dummy Pre-money Valuation",
    "financing_amount": "Dummy Financing Amount",
    "investment_tranches": "Dummy Investment Tranches",
    "key_conditions": "Dummy Key Investment Conditions",
    "equity_shareholding": "Dummy Equity Shareholding Information",
    "governance_terms": "Dummy Governance Terms",
    "liquidation_preferences": "Dummy Liquidation Preferences"
}


In [43]:
# Output in Dataframes
import json
import re
import pandas as pd
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain.output_parsers import OutputFixingParser
from langchain.schema import BaseOutputParser

# ✅ Load Ollama Model
llm = Ollama(model="mistral")  # Ensure correct model

# ✅ Define Dummy Retriever (Replace with actual retriever logic)
def dummy_retriever(_):
    return ["Dummy term sheet text for testing."]

retriever = RunnableLambda(dummy_retriever)

# ✅ Function to Format Retrieved Chunks
def format_docs(docs):
    return "\n\n---\n\n".join(docs)

# ✅ Define Prompt Template
PROMPT_TEMPLATE = """
You are an expert in analyzing financial term sheets. Extract the key investment details in a structured JSON format.

{context}

---

Extract the following details:
- **Company Name**
- **Investor Name**
- **Type of Security** (e.g., Convertible Preference Shares, Equity)
- **Pre-money Valuation**
- **Financing Amount**
- **Investment Tranches** (Number of stages in which funds are disbursed)
- **Key Investment Conditions**
- **Equity Shareholding Information**
- **Voting Rights & Governance Terms**
- **Liquidation Preferences**

Provide ONLY the JSON output, without any explanations.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

# ✅ Define Output Schema
class FinancialTermSheetParser(BaseOutputParser):
    def parse(self, text):
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            raise ValueError("Failed to parse output as JSON.")

output_parser = OutputFixingParser.from_llm(parser=FinancialTermSheetParser(), llm=llm)

# ✅ Define RAG Chain with Forced JSON Output
rag_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),
    }
    | prompt_template
    | llm
    | output_parser  # Forces structured JSON output
)

# ✅ Invoke RAG Chain
response = rag_chain.invoke("Extract key financial details from the term sheet.")

# ✅ Convert to DataFrame
df = pd.DataFrame([response])
print(df)



      Company Name Investor Name               Type of Security  \
0  Example Company  Investor XYZ  Convertible Preference Shares   

   Pre-money Valuation  Financing Amount     Investment Tranches  \
0             10000000           5000000  [Series A (4,000,000)]   

                    Key Investment Conditions  \
0  [Right of First Refusal, Drag-Along Right]   

                     Equity Shareholding Information  \
0  {'Post-Investment Equity Percentage': 30, 'Num...   

                    Voting Rights & Governance Terms  \
0  [Cumulative Voting, Super Majority Vote Required]   

                             Liquidation Preferences  
0  {'Non-Participating Preferred Stock': 1, 'Part...  
